# Install dependencies
- `apache-beam[dask]`: core package being demonstrated here
- `bokeh`: for dask dashboard
- `mimesis`: required for generating example data
- `Pygments`: to `cat` example beam script with syntax highlighting

In [1]:
# !pip install "apache-beam[dask]" "bokeh!=3.0.*,>=2.4.2" mimesis Pygments

Pinning upper bound of `dask` & `distributed` to `2023.9.2` as a workaround until
[this fix](https://github.com/apache/beam/pull/27618/files#diff-bfb5ae715e9067778f492058e8a02ff877d6e7584624908ddbdd316853e6befbL102-R107)
goes in.


In [2]:
# !pip install -U "distributed>=2022.6.0,<2023.9.3"

# Start a client

In [3]:
from distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63834,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63847,Total threads: 2
Dashboard: http://127.0.0.1:63852/status,Memory: 3.20 GiB
Nanny: tcp://127.0.0.1:63837,


# Example data

In [4]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)
dask.datasets.make_people().map(json.dumps).to_textfiles('data/*.json')

['/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/0.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/1.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/2.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/3.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/4.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/5.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/6.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/7.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/8.json',
 '/Users/charlesstern/Dropbox/pangeo/beam-dask-demo/data/9.json']

In [5]:
!head -n 2 data/0.json

{"age": 18, "name": ["Luana", "Dyer"], "occupation": "Insurance Assessor", "telephone": "+1-805-917-0430", "address": {"address": "66 Raycliff Mews", "city": "Folsom"}, "credit-card": {"number": "4439 5434 4716 0773", "expiration-date": "10/21"}}
{"age": 54, "name": ["Domenica", "Vincent"], "occupation": "Pattern Maker", "telephone": "+1-369-692-3326", "address": {"address": "401 Lawton Junction", "city": "Maywood"}, "credit-card": {"number": "4965 9972 5346 9474", "expiration-date": "05/21"}}


In [17]:
!pygmentize -g example.py

import glob
import json
import sys

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.runners.dask.dask_runner import DaskRunner


def yield_jsonlines(fname: str):
    with open(fname) as f:
        for line in f.readlines():
            yield json.loads(line)
        

if __name__ == "__main__":
    opts = dict(runner=DaskRunner(), options=PipelineOptions(sys.argv[1:]))
    with beam.Pipeline(**opts) as p:
        (
            p
            | beam.Create(glob.glob('data/*.json'))
            | beam.FlatMap(yield_jsonlines)
            | beam.Filter(lambda record: record['age'] > 30)
            | beam.Filter(lambda record: record['name'][0].startswith('A'))
            | beam.Filter(lambda record: record['name'][1].startswith('B'))
            | beam.Filter(lambda record: record['occupation'].startswith('C'))
            | beam.Map(lambda record: (" ".join(record['name']), record['age'], record['occupation']))
            | b

In [15]:
!python -W ignore example.py --dask_client_address={client.scheduler.address}

('Alan Benson', 45, 'Cargo Operator')
('Adolph Burton', 31, 'Caretaker')
('Antwan Buckner', 41, 'Car Park Attendant')
('Andree Boyer', 49, 'Caterer')
('Annalisa Blanchard', 60, 'Caulker')
